# Bespoke analyses

Sometimes you may not want to run the DEAGO pipeline in one go.  DEAGO can be broken down into four component scripts:

| Command                      | Description                                                                |
| :--------------------------- | :------------------------------------------------------------------------- |
| **`mart_to_deago`**          | convert delimited annotation for use with DEAGO                            |
| **`build_deago_config`**     | build the DEAGO configuration key/value pairs from command line parameters |
| **`build_deago_markdown`**   | builds markdown from templates but doesn't run the analysis                |
| **`deago_markdown_to_html`** | converts R markdown into HTML report                                       |

The functionality of all of these scripts is incorporated into the runner script: **`deago`**.

The main reason to run the component scripts is when you want to deviate from the standard DEAGO analysis. For example, if you have a two factor analysis where you need to edit the markdown. The examples were designed for the [DEAGO tutorial](DEAGO-tutorial.ipynb) dataset. 

Remember to create a new directory for each analysis:

In [ ]:
    mkdir lrt_analysis && cd lrt_analysis

***

## Converting an annotation

Please see [Preparing an annotation file](Preparing-an-annotation-file.ipynb) for more information on annotation file formats and conversions.

## Building a configuration file

If you want to run an analysis with a bespoke markdown file, you must first build a configuration file (see [Input files](Input-files.ipynb)). DEAGO uses this configuration file to set the analysis parameters (e.g. FDR cutoff) and to determine which templates need to be included (e.g. GO analysis).

In [ ]:
build_deago_config -c ../deago_tutorial/counts \
                   -t ../deago_tutorial/targets.txt \
                   -a ../deago_tutorial/ensembl_mm10_deago_formatted.tsv \
                   --count_type featurecounts \
                   -q 10E-12

This command will build a new configuration file using the command line parameters called [**`deago.config`**](lrt_analysis/deago.config) which looks like:

    annotation_file	/lustre/scratch118/infgen/pathdev/vo1/deago_tutorial/ensembl_mm10_deago_formatted.tsv
    count_column	7
    count_delim	\t
    count_type	featurecounts
    counts_directory	/lustre/scratch118/infgen/pathdev/vo1/deago_tutorial/counts
    gene_ids	Geneid
    go_analysis	0
    go_levels	all
    keep_images	0
    qc_only	0
    qvalue	10E-12
    results_directory	/lustre/scratch118/infgen/pathdev/vo1/deago_tutorial/lrt_analysis
    skip_lines	1
    targets_file	/lustre/scratch118/infgen/pathdev/vo1/deago_tutorial/targets.txt

***

## Build R markdown from templates

The next step in a bespoke analysis is to generate an R markdown file for the standard analysis which can then be edited.  It will take the configuration file that you generated earlier and use the values to define the analysis. 

In [ ]:
    build_deago_markdown -c deago.config

This command gives you your R markdown file [**`deago_markdown.Rmd`**](lrt_analysis/deago_markdown.Rmd), but does not run the analysis.

***

## Editing your markdown

Now you have a markdown file to work with, you can edit the file to run exactly the type of analysis you want.  For example, let's adapt our study design so that it accounts for both cell type and treatment and run a likelihood ratio test (LRT) (see the [DESeq2 vignette](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html) for more information on study design and hypothesis testing).

You can use a command line editor (e.g. `nano`) to edit the markdown file:

In [ ]:
    nano deago_markdown.Rmd

First load the DESeq2 library in your markdown (around line 29) as you will be using functions from the package.

    library(DESeq2)

Now let's edit the study design (this will be around line 60) replacing:

    dds <- runDESeqAnalysis(targets, countdata, parameters)

With the following code:

    coldata <- DataFrame( cell_type=factor( tolower(targets$cell_type) ),
                          treatment=factor( tolower(targets$treatment) ),
                          condition=factor( tolower(targets$condition) ) )

    dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, 
                                  design=~cell_type+treatment+cell_type:treatment)

    dds$cell_type <- relevel(dds$cell_type, ref = "ko")
    dds$treatment <- relevel(dds$treatment, ref = "ctrl")

    dds <- DESeq(dds, test="LRT", reduced=~cell_type+treatment)
    dds <- annotateDataset(dds, parameters)

We can now remove the existing contrast tests, leaving the R session at the end of the analysis and then modify the contrasts to reflect our LRT results:

    lrt_results <- results(dds, alpha=as.numeric(parameters$qvalue))
    lrt_results$symbol <- mcols(dds)$symbol

    contrasts <- list('lrt'=lrt_results)

Finally, replace all occurences of the contrast name **`ko_il22_vs_ko_ctrl`** with **`lrt`**.

Take a look at the [configuration file](lrt_example/deago.config), [modified R markdown file](lrt_example/deago_markdown.Rmd) and [HTML report](lrt_example/deago_markdown.html) that were generated or see the full inputs and results [here](lrt_example).

***

## Converting markdown to HTML

Now you just need to convert your modified markdown into a HTML report.

In [6]:
    deago_markdown_to_html -i deago_markdown.Rmd

This should give you your HTML analysis report: [**`deago_markdown.html`**](lrt_analysis/deago_markdown.html).